In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import os
from processingmm import batch_processing
from histopolalign import combine_images, prepare_images, semi_automatic_tool, align_folders, match_skeletons, align_imgs, finalize_mask, get_numerical_results

# 1. Process the measurements that will be aligned

In [2]:
# get the folder in which the polarimetric measurements are stored
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurements')
neoplastic_folder = '64'
polarimetry_path = os.path.join(neoplastic_polarimetry_path, neoplastic_folder)
directories = [os.path.join(neoplastic_polarimetry_path, neoplastic_folder)]

In [3]:
# process the measurements, if needed
calib_directory = os.path.join(os.getcwd().split('notebooks')[0], 'calib')
batch_processing.batch_process(directories, calib_directory, run_all = False)

0it [00:00, ?it/s]


[]

# 2. Get the combined masks for the histology folders

In [13]:
# create the folde that will be used to align pathology and polarimetry
combine_images.create_to_align_folder()

# load the pathology images...
histology_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HistologyResults')
imgs_all = combine_images.load_the_images(histology_path, Verbose = True)

# ...and process them
combine_images.get_combined_img(imgs_all, force_recompute = True, Verbose = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:01<00:00, 18.64it/s]


Load the images: 1.896 seconds.
Get combined images: 0.366 seconds.


# 3. Obtain the parameters (manually using the GUI) to first align the histology and the polarimetry

### 3.1. Prepare the images

In [14]:
alignment_measurements = prepare_images.create_the_alignments(histology_path, polarimetry_path, Verbose = True)
alignment_measurements

Get polarimetry filenames and rename images: 0.000 seconds.
Create the alignment objects: 0.016 seconds.


[Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\64\2022-11-08_T_HORAO-64-BF_FR_S1_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\64\HORAO-64-S1-1,  status: initialized,
 Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\64\2022-12-06_T_HORAO-64-AF_FR_S1_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\64\HORAO-64-S1-20,  status: initialized]

### 3.2. Load the polarimetry, labels and histology (H&E) images for each polarimetry folder

In [15]:
alignment_measurements = prepare_images.load_and_preprocess_imgs(alignment_measurements, force_recompute = False, Verbose = True)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Get the images and center of mass: 0.020 seconds.


 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  4.08it/s]

Load the images: 0.224 seconds.
Get the images and center of mass: 0.035 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.28it/s]

Load the images: 0.186 seconds.


### 3.3. Semi-automatic rotation/alignement tool

In [17]:
for measurement in alignment_measurements:
    selector = semi_automatic_tool.ask_for_parameters(measurement, force_recompute = False)

# 4. Actually perform the alignment

In [26]:
align_folders.create_align_folders(alignment_measurements, Verbose = True)

 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  2.31it/s]

Process the images to apply the selected changes: 0.297 seconds.
Save the processed images: 0.034 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.25it/s]

Process the images to apply the selected changes: 0.328 seconds.
Save the processed images: 0.032 seconds.


### 4.1. Automatic part

In [27]:
border_parameter = 5
nsamples = 400
max_distance = 150

for measurement in tqdm(alignment_measurements):
    match_skeletons.match_skeletons(measurement, border_parameter, nsamples = nsamples, max_distance = max_distance,
                                                                   Verbose = True)

 50%|██████████████████████████████████████████                                          | 1/2 [00:14<00:14, 14.57s/it]

Match the two skeletons: 14.572 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.31s/it]

Match the two skeletons: 12.053 seconds.


### 4.2. Semi-automatic part

In [28]:
align_folders.semi_automatic_processing(alignment_measurements)

In [29]:
alignment_measurements = align_imgs.align_img_master(alignment_measurements)

### 4.3. Get the final masks and overlay them on the polarimetric parameter maps

In [33]:
finalize_mask.generate_final_masks(alignment_measurements, Verbose = False)

### 4.4. Finalize and save the numerical results

In [34]:
values = get_numerical_results.create_numerical_values(alignment_measurements)